In [4]:
import featureeng as feng
import data.readers as rd
import time
import warnings
from warnings import simplefilter
from featureeng.features import *

warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

instrument = 'EURGBP'
year = 2021

start_time = time.time()
ticks_EURGBP = rd.read_in_ticks('/data', instrument, year)

end_time = time.time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

EURGBP - /data
/data/EURGBP/2021
Execution time: 2.008 seconds


In [5]:
def fill_data(df):
    df['close'] = df['close'].ffill()
    df['close'] = df['close'].bfill()
    
    df['midPoint'] = df['midPoint'].ffill()
    df['midPoint'] = df['midPoint'].bfill()
    
    df['Spread'] = df['Spread'].ffill()
    df['Spread'] = df['Spread'].bfill()
    return df


processed_EURGBP_2 =fill_data(process_df_count(ticks_EURGBP, '5min'))


In [8]:
print(processed_EURGBP_2.shape)
processed_EURGBP_2[0:10]

(104233, 7)


,open,high,low,close,midPoint,Count,Spread
DateTime,,,,,,,
2021-01-03 22:05:00,0.896110,0.896665,0.896110,0.896265,0.896388,53,13.343396
2021-01-03 22:10:00,0.896350,0.896380,0.895975,0.896155,0.896177,82,10.758537
2021-01-03 22:15:00,0.896025,0.896090,0.895820,0.895890,0.895955,16,9.112500
2021-01-03 22:20:00,0.895855,0.896150,0.895835,0.896085,0.895992,18,7.000000
2021-01-03 22:25:00,0.896070,0.896230,0.895425,0.895850,0.895827,58,5.775862
2021-01-03 22:30:00,0.895870,0.896010,0.895400,0.895860,0.895705,46,3.580435
2021-01-03 22:35:00,0.895840,0.896000,0.895670,0.895825,0.895835,154,4.630519
2021-01-03 22:40:00,0.895820,0.895850,0.895730,0.895790,0.895790,117,3.350427
2021-01-03 22:45:00,0.895800,0.895990,0.895780,0.895830,0.895885,65,2.780000


In [12]:
import pandas as pd
import numpy as np
from numba import jit
import time


# Define a custom function to apply over the rolling window

@jit(nopython=True)
def f(window):
    pips = 10000
    # Ensure there are enough data points in the window
    if len(window) != 3:
        return np.nan

    #if increasing
    if window[2][2]-window[0][2]>0:
        return (window[2][1]-window[0][0])*pips
    else:
        return (window[0][1]-window[2][0])*pips
    #if mid-point-to-mid-point is decreasing, then calc gap by C1.L-C3.H
    #Otherwise C3.L-C1.H



start_time = time.time()
# Apply the custom function over a rolling window of size 3 with raw=True
processed_EURGBP_2['gap'] = processed_EURGBP_2[['high', 'low', 'midPoint']].rolling(3, method='table').apply(f, raw=True, engine='numba')['high']

# Print the DataFrame
print(processed_EURGBP_2.between_time('07:00', '17:00'))


end_time = time.time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

                         open      high       low     close  midPoint  Count  \
DateTime                                                                       
2021-01-04 07:00:00  0.895480  0.895680  0.895470  0.895575  0.895575    467   
2021-01-04 07:05:00  0.895570  0.895615  0.895170  0.895440  0.895393    994   
2021-01-04 07:10:00  0.895445  0.895525  0.895305  0.895490  0.895415    609   
2021-01-04 07:15:00  0.895495  0.895545  0.895305  0.895305  0.895425    852   
2021-01-04 07:20:00  0.895300  0.895490  0.895035  0.895415  0.895263    848   
...                       ...       ...       ...       ...       ...    ...   
2021-12-31 16:40:00  0.840080  0.840430  0.840080  0.840295  0.840255    401   
2021-12-31 16:45:00  0.840290  0.840525  0.840290  0.840455  0.840407    355   
2021-12-31 16:50:00  0.840450  0.840710  0.840415  0.840670  0.840563    350   
2021-12-31 16:55:00  0.840675  0.840775  0.840605  0.840665  0.840690    210   
2021-12-31 17:00:00  0.840670  0.840690 

In [13]:
processed_EURGBP_2[['high', 'low', 'midPoint']]

,high,low,midPoint
DateTime,,,
2021-01-03 22:05:00,0.896665,0.896110,0.896388
2021-01-03 22:10:00,0.896380,0.895975,0.896177
2021-01-03 22:15:00,0.896090,0.895820,0.895955
2021-01-03 22:20:00,0.896150,0.895835,0.895992
2021-01-03 22:25:00,0.896230,0.895425,0.895827
...,...,...,...
2021-12-31 19:45:00,0.841030,0.840965,0.840997
2021-12-31 19:50:00,0.841080,0.840855,0.840967
2021-12-31 19:55:00,0.841135,0.840885,0.841010
